In [1]:
!rm -rf /kaggle/working/ds201_BTTH2
!git clone https://github.com/UngHoangLong/ds201_BTTH2.git

Cloning into 'ds201_BTTH2'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 54 (delta 8), reused 11 (delta 4), pack-reused 39 (from 1)
Receiving objects: 100% (54/54), 61.02 MiB | 43.51 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [2]:
import torch
import torch.nn as nn
from torch.optim import Adam
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import sys

# Import các thư viện HuggingFace
from transformers import AutoImageProcessor, ResNetForImageClassification

# Thêm đường dẫn (path) (Hãy đảm bảo đường dẫn này đúng)
sys.path.append("/kaggle/working/ds201_BTTH2") 

# --- THAY ĐỔI: Import hàm mới của bạn ---
from data_loader import get_vinfood_dataloaders_for_HuggingFace

print("Các thư viện và module đã được tải.")

2025-11-06 10:26:00.098613: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762424760.121509     174 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762424760.128263     174 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Các thư viện và module đã được tải.


In [3]:
# Cấu hình data
BATCH_SIZE = 32 # ResNet-50 nặng, nên dùng batch size 32 hoặc 16
model_name="microsoft/resnet-50" # Định nghĩa tên model ở đây

# (Hãy đảm bảo đường dẫn này đúng với Kaggle)
train_data_path = "/kaggle/input/vinafood21/VinaFood21/train"
test_data_path = "/kaggle/input/vinafood21/VinaFood21/test"

# --- THAY ĐỔI: Gọi hàm mới và nhận 6 giá trị ---
train_loader, val_loader, test_loader, NUM_CLASSES, label2id, id2label = \
    get_vinfood_dataloaders_for_HuggingFace(
        model_name=model_name,
        batch_size=BATCH_SIZE, 
        train_path=train_data_path, 
        test_path=test_data_path, 
        val_split=0.2
    )

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Đã tải data (chuẩn HF ResNet-50) thành công. Tổng cộng 21 lớp.
Train: 8035 ảnh | Validation: 2009 ảnh (đã chia stratified).


In [4]:
# Cấu hình
LEARNING_RATE = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

# --- THAY ĐỔI: Tải mô hình Pretrained (Cách chuẩn) ---
# Chúng ta truyền số lớp và mapping vào thẳng
model = ResNetForImageClassification.from_pretrained(
    model_name,
    num_labels=NUM_CLASSES, # <--- Tự động thay thế lớp cuối
    label2id=label2id,      # <--- Cung cấp mapping
    id2label=id2label,      # <--- Cung cấp mapping
    ignore_mismatched_sizes=True # Báo: "Tôi biết lớp cuối (1000) và (21)
                                 # không khớp, hãy vứt bỏ lớp cũ"
)

print(f"Đã tải mô hình gốc {model_name} và thay thế đầu classifier.")

# --- 2. Đóng băng (Freeze) toàn bộ các lớp gốc ---
# (Phần thân của ResNet trong transformers được gọi là 'resnet')
for param in model.resnet.parameters():
    param.requires_grad = False # Báo cho PyTorch: "Đừng tính gradient"

print("Đã đóng băng (freeze) các lớp ResNet gốc.")

# --- 3. Chuyển mô hình sang GPU ---
model = model.to(device)

# --- 4. Thiết lập Optimizer ---
# Chỉ đưa các tham số của LỚP MỚI (classifier) vào optimizer
optimizer = Adam(model.classifier.parameters(), lr=LEARNING_RATE)

# Hàm loss
criterion = nn.CrossEntropyLoss() # (Mặc dù model tự tính loss, ta vẫn
                                # có thể cần nó nếu muốn)

print("Đã khởi tạo model, criterion, và optimizer (chỉ cho lớp classifier).")

Sử dụng thiết bị: cuda


Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-50 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([21]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([21, 2048]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Đã tải mô hình gốc microsoft/resnet-50 và thay thế đầu classifier.
Đã đóng băng (freeze) các lớp ResNet gốc.
Đã khởi tạo model, criterion, và optimizer (chỉ cho lớp classifier).


In [5]:
NUM_EPOCHS = 20 # Fine-tuning không cần nhiều epoch
history = {'train_loss': [], 'val_f1': [], 'val_precision': [], 'val_recall': []}

best_val_f1 = 0.0
MODEL_SAVE_PATH = "best_resnet50_finetuned_model.pth"

print(f'Bắt đầu quá trình fine-tuning cho {NUM_EPOCHS} epochs...')
for epoch in range(NUM_EPOCHS):
    
    # --- Training ---
    model.train()
    running_loss = 0.0
    train_progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{NUM_EPOCHS} - Train', leave=False)
    
    for images, labels in train_progress_bar:
        
        if images is None or labels is None:
            print('Lỗi ảnh')
            continue # Bỏ qua batch hỏng này
            
        images, labels = images.to(device), labels.to(device)

        # Mô hình HF tự động tính loss khi có 'labels'
        output = model(pixel_values=images, labels=labels)
        
        # Lấy loss trực tiếp từ output
        loss = output.loss 

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        train_progress_bar.set_postfix(batch_loss=loss.item())

    epoch_train_loss = running_loss / len(train_loader)
    history['train_loss'].append(epoch_train_loss)

    # --- Validation ---
    model.eval()
    val_preds=[]
    val_labels=[]
    val_progress_bar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{NUM_EPOCHS} - Val', leave=False)
    
    with torch.no_grad():
        for images, labels in val_progress_bar:
            images, labels = images.to(device), labels.to(device)
            
            # Không đưa labels khi eval
            output = model(pixel_values=images) 
            
            # Logits (điểm số thô) nằm trong .logits
            _, predicted = torch.max(output.logits.data, 1) 

            val_preds.extend(predicted.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())


    precision = precision_score(val_labels, val_preds, average='macro', zero_division=0)
    recall = recall_score(val_labels, val_preds, average='macro', zero_division=0)
    f1 = f1_score(val_labels, val_preds, average='macro', zero_division=0)
    
    history['val_precision'].append(precision)
    history['val_recall'].append(recall)
    history['val_f1'].append(f1)
    
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Train Loss: {epoch_train_loss:.4f} | Val F1 (Macro): {f1:.4f}")

    if f1 > best_val_f1:
        best_val_f1 = f1
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print(f"  -> Đã lưu model tốt nhất mới! F1: {best_val_f1:.4f} tại {MODEL_SAVE_PATH}")

print(f"\nHuấn luyện hoàn tất! F1 tốt nhất trên Validation là: {best_val_f1:.4f}")

Bắt đầu quá trình fine-tuning cho 20 epochs...


Epoch 1/20 - Train:   0%|          | 0/252 [00:00<?, ?it/s]

Epoch 1/20 - Val:   0%|          | 0/63 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/20 - Train Loss: 2.4715 | Val F1 (Macro): 0.4872
  -> Đã lưu model tốt nhất mới! F1: 0.4872 tại best_resnet50_finetuned_model.pth


...

Epoch 19/20 - Val:   0%|          | 0/63 [00:00<?, ?it/s]

Epoch 19/20 - Train Loss: 0.6519 | Val F1 (Macro): 0.6994


Epoch 20/20 - Train:   0%|          | 0/252 [00:00<?, ?it/s]

Epoch 20/20 - Val:   0%|          | 0/63 [00:00<?, ?it/s]

Epoch 20/20 - Train Loss: 0.6365 | Val F1 (Macro): 0.6912

Huấn luyện hoàn tất! F1 tốt nhất trên Validation là: 0.7044


In [8]:
print("Đang đánh giá kết quả cuối cùng trên tập Test...")

# 1. Khởi tạo kiến trúc ResNet-50 với đúng số lớp (21)
# (Chúng ta cần truyền label2id và id2label y hệt lúc train)
test_model = ResNetForImageClassification.from_pretrained(
    model_name, 
    num_labels=NUM_CLASSES,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True
).to(device)

# 2. Tải trọng số (weights) đã lưu
test_model.load_state_dict(torch.load(MODEL_SAVE_PATH))
test_model.eval()


test_preds = []
test_labels = []
test_progress_bar = tqdm(test_loader, desc="Testing", leave=False)

with torch.no_grad():
    for images, labels in test_progress_bar:
        images, labels = images.to(device), labels.to(device)
        output = test_model(pixel_values=images) 
        _, predicted = torch.max(output.logits.data, 1)
        test_preds.extend(predicted.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

# Tính toán các độ đo cuối cùng
test_precision = precision_score(test_labels, test_preds, average='macro', zero_division=0)
test_recall = recall_score(test_labels, test_preds, average='macro', zero_division=0)
test_f1 = f1_score(test_labels, test_preds, average='macro', zero_division=0)

print("\n--- KẾT QUẢ CUỐI CÙNG TRÊN TẬP TEST (từ model tốt nhất) ---")
print(f"Test Precision (Macro): {test_precision:.4f}")
print(f"Test Recall (Macro):    {test_recall:.4f}")
print(f"Test F1-Score (Macro):  {test_f1:.4f}")

Đang đánh giá kết quả cuối cùng trên tập Test...


Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-50 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([21]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([21, 2048]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Testing:   0%|          | 0/209 [00:00<?, ?it/s]


--- KẾT QUẢ CUỐI CÙNG TRÊN TẬP TEST (từ model tốt nhất) ---
Test Precision (Macro): 0.6633
Test Recall (Macro):    0.6563
Test F1-Score (Macro):  0.6503
